In [1]:
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import Descriptors
from rdkit.Chem.rdMolDescriptors import CalcNumHBA
from rdkit.Chem.rdMolDescriptors import CalcNumHBD

import numpy as np
import pandas as pd

import os

## bring DB --> primary processing

In [2]:
a=pd.read_excel('../collecting/hand_minhi.xlsx' , header = None) # hand_minhi.xslsx 로 이름 변경
b=pd.read_excel('../collecting/hand_seokwoo.xlsm', header = None)  # hand_seokwoo.xlsm 로 이름 변경경

In [3]:
b

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,NaN,Molecule,NaN,NaN,Eluent,NaN,NaN,TLC plates,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Acetone
1,Index,Name,SMILES,staining solution,eluent1,eluent2,ratio,Base material,Surface,Thickness,...,"year,vol,issue",PE,alter,elu1_smile,elu2_smile,NaN,NaN,NaN,NaN,Chloroform
2,1,Enone4,C=CCC1OC2COC(C)(OC2CC1OCC(C=C)=O)C,NaN,diethyl ether,PE,3:2,aluminium,silica,NaN,...,"2024,26,4",40−60 °C,pentane,NaN,NaN,NaN,NaN,NaN,NaN,DCM
3,2,Allylic Alcohol 5,C=CCC2OC1COC(C)(C)OC1CC2OCC(O)CC,NaN,diethyl ether,PE,4:1,aluminium,silica,NaN,...,NaN,40−60 °C,pentane,NaN,NaN,NaN,NaN,NaN,NaN,EA
4,3,Enone 6,CC3(C)OCC2OC1C/C=C\C(=O)COC1CC2O3,NaN,diethyl ether,PE,4:1,aluminium,silica,NaN,...,NaN,40−60 °C,pentane,NaN,NaN,NaN,NaN,NaN,NaN,Hex
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3515,3514,4p,CC[C@H](NC(C)=O)C1=CC=C(COC2=CC3=C(C=C2)[C@H]2...,NaN,EA,PE,1:1,aluminum,silica,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3516,3515,5,CCOC(=O)CCC1(CCCCC1)NC(C)=O,NaN,EA,PE,1:1,aluminum,silica,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3517,3516,6,O=C1CCC2(CCCCC2)N1,NaN,EA,PE,1:1,aluminum,silica,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3518,3517,7,CCC(NC(C)=O)C1=CC=C(CN2CCN(CC2)C2=CC=CC=C2)C=C1,NaN,EA,PE,7:3,aluminum,silica,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# 데이터 합치기

a= a.drop([0,1])
a=a.iloc[:, [2,4,5,6,12,13,14]]
a=a[~a.iloc[:,0].isna()] # compound 안적혀 있는 거 제거
a=a[~a[14].isna()]  # Rf 안적혀 있는 거 제거
a= a.dropna(subset = [4,5], how = 'all') # eluent 두개 모두 없는거 제거

b= b.drop([0,1])
b=b.iloc[:, [2,4,5,6,12,13,14]]
b=b[~b.iloc[:,0].isna()]
b=b[~b[14].isna()]
b= b.dropna(subset = [4,5], how = 'all')

raw = pd.concat([a,b], axis=0)
raw.index = range(len(raw))
raw.columns=['Smiles','elu1','elu2','ratio', 'doi', 'title', 'Rf']

In [4]:
raw

,Smiles,elu1,elu2,ratio,doi,title,Rf
0,BrC1=CC=C(CC(C(CC)=O)C(CC)=O)C=C1,EA,Hex,1:9,10.1021/acs.orglett.3c03369,A Site-Specific Synthetic Route to Substituted...,0.4
1,BrC1=CC=C(CC(C(O)=O)C(O)=O)C=C1,EA,Hex,3:7,10.1021/acs.orglett.3c03369,A Site-Specific Synthetic Route to Substituted...,0.05
2,BrC1=CC=C(CCC(O)=O)C=C1,EA,Hex,3:7,10.1021/acs.orglett.3c03369,A Site-Specific Synthetic Route to Substituted...,0.1
3,BrC1=CC=C2C(C(CC2)=O)=C1,EA,Hex,3:7,10.1021/acs.orglett.3c03369,A Site-Specific Synthetic Route to Substituted...,0.65
4,BrC1=CC=C2C(C(O)CC2)=C1,EA,Hex,3:7,10.1021/acs.orglett.3c03369,A Site-Specific Synthetic Route to Substituted...,0.55
...,...,...,...,...,...,...,...
4670,CC[C@H](NC(C)=O)C1=CC=C(COC2=CC3=C(C=C2)[C@H]2...,EA,PE,1:1,https://pubs.acs.org/doi/10.1021/acs.orglett.5...,Photochemical Alkylamination of Olefins throug...,0.3
4671,CCOC(=O)CCC1(CCCCC1)NC(C)=O,EA,PE,1:1,https://pubs.acs.org/doi/10.1021/acs.orglett.5...,Photochemical Alkylamination of Olefins throug...,0.4
4672,O=C1CCC2(CCCCC2)N1,EA,PE,1:1,https://pubs.acs.org/doi/10.1021/acs.orglett.5...,Photochemical Alkylamination of Olefins throug...,0.4
4673,CCC(NC(C)=O)C1=CC=C(CN2CCN(CC2)C2=CC=CC=C2)C=C1,EA,PE,7:3,https://pubs.acs.org/doi/10.1021/acs.orglett.5...,Photochemical Alkylamination of Olefins throug...,0.3


In [5]:
# eluent --> key --> smiles 변환

elu_to_key_df = pd.read_csv('elu_to_key_df.csv') 
elu_to_key_converter = dict(zip(elu_to_key_df['elu_raw'], elu_to_key_df['elu_key']))

elu_df = pd.read_csv('eluent_standard.csv')
key_to_567smiles_converter = dict(zip(elu_df['key'], elu_df['567_Smiles']))
key_to_smiles_converter = dict(zip(elu_df['key'], elu_df['hex_Smiles']))

elu1_list_org = raw['elu1']
elu2_list_org = raw['elu2']

# -----------PE --> hex 버전 ---------------

raw['elu1'] = [key_to_smiles_converter[elu_to_key_converter[name]] for name in elu1_list_org]
raw['elu2']  = [key_to_smiles_converter[elu_to_key_converter[name]] for name in elu2_list_org]

# ratio 칼럼 정리
ratio1 = [float(rat.split(':')[0]) / (float(rat.split(':')[0]) + float(rat.split(':')[1]))
          if isinstance(rat, str) and  ':' in rat else float(1)  for rat in raw['ratio']]
ratio2 = [float(rat.split(':')[1]
) / (float(rat.split(':')[0]) + float(rat.split(':')[1]))
          if isinstance(rat, str) and ':' in rat else float(0)  for rat in raw['ratio']]
raw['ratio1'] = ratio1
raw['ratio2'] = ratio2
raw.drop(columns = ['ratio'], inplace=True)
raw = raw[['Smiles', 'elu1', 'elu2', 'ratio1', 'ratio2', 'Rf', 'title', 'doi']]

raw['ratio1'] = raw['ratio1'].apply(lambda x: float(x) if not isinstance(x, float) else x)
raw['ratio2'] = raw['ratio2'].apply(lambda x: float(x) if not isinstance(x, float) else x)
raw['Rf'] = raw['Rf'].apply(lambda x: float(x) if not isinstance(x, float) else x)

#----------PE --> 567버전 ------------------
from copy import deepcopy
raw_567 = deepcopy(raw)
raw_567['elu1'] = [key_to_567smiles_converter[elu_to_key_converter[name]] for name in elu1_list_org]
raw_567['elu2']  = [key_to_567smiles_converter[elu_to_key_converter[name]] for name in elu2_list_org]

In [6]:
import sys
sys.path.append('../../../')
from functions import delete_weird_mol_and_None_except_Nan, delete_metal_mol_except_Nan


# ---------------PE_hex 버전

print('1')
raw = delete_weird_mol_and_None_except_Nan(raw, 'Smiles')
print('2')
raw = delete_metal_mol_except_Nan(raw, 'Smiles')
print('3')
raw = delete_weird_mol_and_None_except_Nan(raw, 'elu1')
print('4')
raw = delete_metal_mol_except_Nan(raw, 'elu1')
print('5')
raw = delete_weird_mol_and_None_except_Nan(raw, 'elu2')
print('6')
raw = delete_metal_mol_except_Nan(raw, 'elu2')
print('7')


# ---------------PE_567 버전

print('1')
raw_567 = delete_weird_mol_and_None_except_Nan(raw_567, 'Smiles')
print('2')
raw_567 = delete_metal_mol_except_Nan(raw_567, 'Smiles')
print('3')
raw_567 = delete_weird_mol_and_None_except_Nan(raw_567, 'elu1')
print('4')
raw_567 = delete_metal_mol_except_Nan(raw_567, 'elu1')
print('5')
raw_567 = delete_weird_mol_and_None_except_Nan(raw_567, 'elu2')
print('6')
raw_567 = delete_metal_mol_except_Nan(raw_567, 'elu2')
print('7')

1
11번째 분자 mol=None (smiles가 아닌 값)
153번째 분자 mol=None (smiles가 아닌 값)
174번째 분자 mol=None (smiles가 아닌 값)


[21:06:00] Explicit valence for atom # 10 Br, 3, is greater than permitted
[21:06:00] Explicit valence for atom # 20 F, 3, is greater than permitted
[21:06:00] Explicit valence for atom # 12 F, 3, is greater than permitted
[21:06:00] Conflicting single bond directions around double bond at index 7.
[21:06:00]   BondStereo set to STEREONONE and single bond directions set to NONE.
[21:06:00] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 6 8
[21:06:00] SMILES Parse Error: syntax error while parsing: Methyl
[21:06:00] SMILES Parse Error: Failed parsing SMILES 'Methyl' for input: 'Methyl'
[21:06:00] SMILES Parse Error: syntax error while parsing: Methyl
[21:06:00] SMILES Parse Error: Failed parsing SMILES 'Methyl' for input: 'Methyl'
[21:06:00] SMILES Parse Error: syntax error while parsing: (2-hydroxy-5-iodobenzoyl)proline
[21:06:00] SMILES Parse Error: Failed parsing SMILES '(2-hydroxy-5-iodobenzoyl)proline' for input: '(2-hydroxy-5-iodobenzoyl)proline'
[21:06:00] SMILES Parse Error: 

1336번째 분자 mol=None (smiles가 아닌 값)
1699번째 분자 mol=None (smiles가 아닌 값)
1708번째 분자 mol=None (smiles가 아닌 값)
1723번째 분자 mol=None (smiles가 아닌 값)
1782번째 분자 mol=None (smiles가 아닌 값)
1807번째 분자 mol=None (smiles가 아닌 값)
1808번째 분자 mol=None (smiles가 아닌 값)
1844번째 분자 mol=None (smiles가 아닌 값)
2083번째 분자 mol=None (smiles가 아닌 값)


[21:06:00] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5
[21:06:00] SMILES Parse Error: syntax error while parsing: (2-Fluoro-5-phenylpenta-2,4-dien-1-yl)-2,5-dimethyl-2H-pyrrole
[21:06:00] SMILES Parse Error: Failed parsing SMILES '(2-Fluoro-5-phenylpenta-2,4-dien-1-yl)-2,5-dimethyl-2H-pyrrole' for input: '(2-Fluoro-5-phenylpenta-2,4-dien-1-yl)-2,5-dimethyl-2H-pyrrole'
[21:06:00] SMILES Parse Error: syntax error while parsing: tert-butyl
[21:06:00] SMILES Parse Error: Failed parsing SMILES 'tert-butyl' for input: 'tert-butyl'
[21:06:00] SMILES Parse Error: syntax error while parsing: tert-butyl
[21:06:00] SMILES Parse Error: Failed parsing SMILES 'tert-butyl' for input: 'tert-butyl'
[21:06:00] SMILES Parse Error: syntax error while parsing: 2,2'-(2-(3,4-dichlorophenyl)ethane-1,1-diyl)bis(4,4,5,5-tetramethyl-1,3,2-dioxaborolane)
[21:06:00] SMILES Parse Error: Failed parsing SMILES '2,2'-(2-(3,4-dichlorophenyl)ethane-1,1-diyl)bis(4,4,5,5-tetramethyl-1,3,2-dioxaborolane)' for input: 

3280번째 분자 mol=None (smiles가 아닌 값)
3281번째 분자 mol=None (smiles가 아닌 값)
3512번째 분자 mol=None (smiles가 아닌 값)
3513번째 분자 mol=None (smiles가 아닌 값)
3975번째 분자 mol=None (smiles가 아닌 값)
4063번째 분자 mol=None (smiles가 아닌 값)


...

3280번째 분자 mol=None (smiles가 아닌 값)
3281번째 분자 mol=None (smiles가 아닌 값)
3512번째 분자 mol=None (smiles가 아닌 값)
3513번째 분자 mol=None (smiles가 아닌 값)
3975번째 분자 mol=None (smiles가 아닌 값)
4063번째 분자 mol=None (smiles가 아닌 값)


[21:06:04] SMILES Parse Error: syntax error while parsing: 2-((3R,5S,6R,7R,E)-10-(tert-butyldimethylsilyloxy)-5-methoxy-7-(methoxymethoxy)-3,9-dimethyl-6-(triethylsilyloxy)dec-8-enyl)-4-((4S,5S,E)-4-(4-methoxybenzyloxy)-5-methyl-6-(triisopropylsilyloxy)hex-2-en-2-yl)oxazole
[21:06:04] SMILES Parse Error: Failed parsing SMILES '2-((3R,5S,6R,7R,E)-10-(tert-butyldimethylsilyloxy)-5-methoxy-7-(methoxymethoxy)-3,9-dimethyl-6-(triethylsilyloxy)dec-8-enyl)-4-((4S,5S,E)-4-(4-methoxybenzyloxy)-5-methyl-6-(triisopropylsilyloxy)hex-2-en-2-yl)oxazole' for input: '2-((3R,5S,6R,7R,E)-10-(tert-butyldimethylsilyloxy)-5-methoxy-7-(methoxymethoxy)-3,9-dimethyl-6-(triethylsilyloxy)dec-8-enyl)-4-((4S,5S,E)-4-(4-methoxybenzyloxy)-5-methyl-6-(triisopropylsilyloxy)hex-2-en-2-yl)oxazole'


4392번째 분자 mol=None (smiles가 아닌 값)
2


[21:06:04] Conflicting single bond directions around double bond at index 7.
[21:06:04]   BondStereo set to STEREONONE and single bond directions set to NONE.
[21:06:04] Conflicting single bond directions around double bond at index 7.
[21:06:04]   BondStereo set to STEREONONE and single bond directions set to NONE.


2697번째 분자 금속원소 포함됨
2698번째 분자 금속원소 포함됨
2700번째 분자 금속원소 포함됨
2701번째 분자 금속원소 포함됨
2704번째 분자 금속원소 포함됨
2705번째 분자 금속원소 포함됨
2708번째 분자 금속원소 포함됨
2709번째 분자 금속원소 포함됨
4347번째 분자 금속원소 포함됨
4348번째 분자 금속원소 포함됨
4349번째 분자 금속원소 포함됨
4353번째 분자 금속원소 포함됨
4363번째 분자 금속원소 포함됨
4366번째 분자 금속원소 포함됨
4370번째 분자 금속원소 포함됨
4371번째 분자 금속원소 포함됨
4372번째 분자 금속원소 포함됨
3
4
5
6
7


In [7]:
#Smiles canonicalize

#-------------PE hex 버전 -------------

raw = raw[(0<raw['Rf']) & (raw['Rf']<=1)]

# ratio1 > ratio2 되도록 정렬 --> 중복 데이터 판별시 필요
switch_ratio_mask = raw['ratio1']-raw['ratio2']<0

raw.loc[switch_ratio_mask, ['ratio1','ratio2']] = raw.loc[switch_ratio_mask, ['ratio2', 'ratio1']].values
raw.loc[switch_ratio_mask, ['elu1','elu2']] = raw.loc[switch_ratio_mask, ['elu2', 'elu1']].values

# compound smiles canonicalize 
raw.loc[:,'Smiles'] = [Chem.MolToSmiles(Chem.MolFromSmiles(smiles), isomericSmiles=True, canonical=True) for smiles in raw['Smiles']]

raw = raw.reset_index(drop=True)


#-------------PE 567 버전 -------------

raw_567 = raw_567[(0<raw_567['Rf']) & (raw_567['Rf']<=1)]

# ratio1 > ratio2 되도록 정렬 --> 중복 데이터 판별시 필요
switch_ratio_mask = raw_567['ratio1']-raw_567['ratio2']<0

raw_567.loc[switch_ratio_mask, ['ratio1','ratio2']] = raw_567.loc[switch_ratio_mask, ['ratio2', 'ratio1']].values
raw_567.loc[switch_ratio_mask, ['elu1','elu2']] = raw_567.loc[switch_ratio_mask, ['elu2', 'elu1']].values

# compound smiles canonicalize 
raw_567.loc[:,'Smiles'] = [Chem.MolToSmiles(Chem.MolFromSmiles(smiles), isomericSmiles=True, canonical=True) for smiles in raw_567['Smiles']]

raw_567 = raw_567.reset_index(drop=True)

[21:06:07] Conflicting single bond directions around double bond at index 7.
[21:06:07]   BondStereo set to STEREONONE and single bond directions set to NONE.
[21:06:08] Conflicting single bond directions around double bond at index 7.
[21:06:08]   BondStereo set to STEREONONE and single bond directions set to NONE.


### elu1,elu2 도 canonlicalize 완료!

In [8]:
raw['elu1'] = [Chem.MolToSmiles(Chem.MolFromSmiles(s), canonical=True) if pd.notna(s) else np.nan for s in raw['elu1']]
raw['elu2'] = [Chem.MolToSmiles(Chem.MolFromSmiles(s), canonical=True) if pd.notna(s) else np.nan for s in raw['elu2']]

raw_567['elu1'] = [Chem.MolToSmiles(Chem.MolFromSmiles(s), canonical=True) if pd.notna(s) else np.nan for s in raw_567['elu1']]
raw_567['elu2'] = [Chem.MolToSmiles(Chem.MolFromSmiles(s), canonical=True) if pd.notna(s) else np.nan for s in raw_567['elu2']]

### Rf_type 추가

In [9]:
def set_Rf_type(df, not_005_ratio=0.5):
    from decimal import Decimal

    C_Rf_paper_list=[]
    for paper, paper_data in df.groupby(['title'], as_index=True):
        is_it_precise = np.array([Decimal(str(rf)) for rf in paper_data['Rf']]) % Decimal('0.05')
        # if (is_it_precise > 0).any():
        if len(is_it_precise[is_it_precise>0]) > len(is_it_precise)*not_005_ratio:
            C_Rf_paper_list.append(paper[0])

    df['Rf_type'] = ['C' if (title in C_Rf_paper_list) else 'Q' for title in df['title']]
    return df

In [10]:
raw_final = set_Rf_type(raw, 0.7)
raw_567_final = set_Rf_type(raw_567, 0.7)

In [11]:
raw_final

,Smiles,elu1,elu2,ratio1,ratio2,Rf,title,doi,Rf_type
0,CCC(=O)C(Cc1ccc(Br)cc1)C(=O)CC,CCCCCC,CCOC(C)=O,0.9,0.1,0.40,A Site-Specific Synthetic Route to Substituted...,10.1021/acs.orglett.3c03369,Q
1,O=C(O)C(Cc1ccc(Br)cc1)C(=O)O,CCCCCC,CCOC(C)=O,0.7,0.3,0.05,A Site-Specific Synthetic Route to Substituted...,10.1021/acs.orglett.3c03369,Q
2,O=C(O)CCc1ccc(Br)cc1,CCCCCC,CCOC(C)=O,0.7,0.3,0.10,A Site-Specific Synthetic Route to Substituted...,10.1021/acs.orglett.3c03369,Q
3,O=C1CCc2ccc(Br)cc21,CCCCCC,CCOC(C)=O,0.7,0.3,0.65,A Site-Specific Synthetic Route to Substituted...,10.1021/acs.orglett.3c03369,Q
4,OC1CCc2ccc(Br)cc21,CCCCCC,CCOC(C)=O,0.7,0.3,0.55,A Site-Specific Synthetic Route to Substituted...,10.1021/acs.orglett.3c03369,Q
...,...,...,...,...,...,...,...,...,...
4633,CC[C@H](NC(C)=O)c1ccc(COc2ccc3c(c2)CC[C@@H]2[C...,CCOC(C)=O,CCCCCC,0.5,0.5,0.30,Photochemical Alkylamination of Olefins throug...,https://pubs.acs.org/doi/10.1021/acs.orglett.5...,Q
4634,CCOC(=O)CCC1(NC(C)=O)CCCCC1,CCOC(C)=O,CCCCCC,0.5,0.5,0.40,Photochemical Alkylamination of Olefins throug...,https://pubs.acs.org/doi/10.1021/acs.orglett.5...,Q
4635,O=C1CCC2(CCCCC2)N1,CCOC(C)=O,CCCCCC,0.5,0.5,0.40,Photochemical Alkylamination of Olefins throug...,https://pubs.acs.org/doi/10.1021/acs.orglett.5...,Q
4636,CCC(NC(C)=O)c1ccc(CN2CCN(c3ccccc3)CC2)cc1,CCOC(C)=O,CCCCCC,0.7,0.3,0.30,Photochemical Alkylamination of Olefins throug...,https://pubs.acs.org/doi/10.1021/acs.orglett.5...,Q


In [12]:
raw_final.to_csv('../collecting/primary_processed_PE_hex/hand_PE_hex.csv', index=False)
raw_567_final.to_csv('../collecting/primary_processed_PE_567/hand_PE_567.csv', index=False)